In [1]:
pip install tensorflow numpy matplotlib pillow

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.3 -> 26.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
# ==============================
# 1. IMPORT LIBRARIES
# ==============================
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array

# ==============================
# 2. PARAMETERS
# ==============================
IMG_HEIGHT = 128
IMG_WIDTH = 128
BATCH_SIZE = 16
EPOCHS = 2           # You can increase to 20-30 for better accuracy
MODEL_PATH = "dog_cat_cnn_model.h5"
TEST_IMAGE = "dataset/image.png"  # Your test image

# ==============================
# 3. DATA GENERATORS
# ==============================
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    zoom_range=0.2,
    horizontal_flip=True
)

validation_datagen = ImageDataGenerator(rescale=1./255)

train_data = train_datagen.flow_from_directory(
    "dataset/train",        # train folder
    target_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=BATCH_SIZE,
    class_mode="binary"
)

validation_data = validation_datagen.flow_from_directory(
    "dataset/validation",   # validation folder
    target_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=BATCH_SIZE,
    class_mode="binary"
)

# ==============================
# 4. CNN MODEL
# ==============================
model = Sequential([
    Conv2D(32, (3,3), activation='relu', input_shape=(IMG_HEIGHT, IMG_WIDTH, 3)),
    MaxPooling2D(2,2),
    Conv2D(64, (3,3), activation='relu'),
    MaxPooling2D(2,2),
    Conv2D(128, (3,3), activation='relu'),
    MaxPooling2D(2,2),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')
])

model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['accuracy']
)

model.summary()

# ==============================
# 5. TRAIN MODEL
# ==============================
history = model.fit(
    train_data,
    epochs=EPOCHS,
    validation_data=validation_data
)

# ==============================
# 6. SAVE MODEL
# ==============================
model.save(MODEL_PATH)
print(f"Model saved as {MODEL_PATH}")

# ==============================
# 7. PREDICT FUNCTION
# ==============================
def predict_image(image_path):
    if not os.path.exists(MODEL_PATH):
        print("Model not found! Train the model first.")
        return

    model = load_model(MODEL_PATH)

    if not os.path.exists(image_path):
        print(f"Image '{image_path}' not found! Put it in the dataset folder.")
        return

    img = load_img(image_path, target_size=(IMG_HEIGHT, IMG_WIDTH))
    img_array = img_to_array(img) / 255.0
    img_array = np.expand_dims(img_array, axis=0)

    prediction = model.predict(img_array)
    if prediction[0][0] > 0.5:
        print(f"{image_path} → Dog ({prediction[0][0]:.2f})")
    else:
        print(f"{image_path} → Cat ({1 - prediction[0][0]:.2f})")

# ==============================
# 8. RUN PREDICTION
# ==============================
predict_image(TEST_IMAGE)

Found 1000 images belonging to 2 classes.
Found 202 images belonging to 2 classes.


Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_6 (Conv2D)               │ (None, 126, 126, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_6 (MaxPooling2D)  │ (None, 63, 63, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_7 (Conv2D)               │ (None, 61, 61, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_7 (MaxPooling2D)  │ (None, 30, 30, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_8 (Conv2D)               │ (None, 28, 28, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_8 (MaxPooling2D)  │ (None, 14, 14, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_2 (Flatten)             │ (None, 25088)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 128)            │     3,211,392 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 1)              │           129 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 3,304,769 (12.61 MB)

 Trainable params: 3,304,769 (12.61 MB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/2
63/63 ━━━━━━━━━━━━━━━━━━━━ 39s 608ms/step - accuracy: 0.6370 - loss: 0.6105 - val_accuracy: 0.8020 - val_loss: 0.4395
Epoch 2/2
63/63 ━━━━━━━━━━━━━━━━━━━━ 33s 515ms/step - accuracy: 0.8540 - loss: 0.3898 - val_accuracy: 0.9356 - val_loss: 0.1784


Model saved as dog_cat_cnn_model.h5


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 217ms/step
dataset/image.png → Cat (0.94)
